In [1]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install imutils


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install pytesseract


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import cv2
import imutils
import pytesseract

In [5]:
pytesseract.pytesseract.tesseract_cmd = r"C:/Users/avish/Desktop/Tesseract-OCR/Tesseract-OCR"

In [2]:
import cv2
import imutils
import pytesseract
import numpy as np
from tkinter import Tk, Button, Label, filedialog
from PIL import Image, ImageTk

# ✅ Path to tesseract.exe (only for Windows)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Tkinter root
root = Tk()
root.title("License Plate Detection")
root.geometry("800x600")

# Label for images
image_label = Label(root)
image_label.pack()

# Label for detected text
result_label = Label(root, text="Detected Plate: ", font=("Arial", 16))
result_label.pack(pady=10)


def upload_and_detect():
    file_path = filedialog.askopenfilename(
        title="Select a car image",
        filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")]
    )
    
    if not file_path:
        return
    
    # Load image
    image = cv2.imread(file_path)
    image = imutils.resize(image, width=500)
    orig = image.copy()

    # Convert to gray
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)

    # Edge detection
    edged = cv2.Canny(gray, 30, 200)

    # Find contours
    cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:30]

    screenCnt = None
    plate_text = "Not detected"

    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)

        if len(approx) == 4:  # Likely license plate
            screenCnt = approx
            x, y, w, h = cv2.boundingRect(c)
            new_img = image[y:y+h, x:x+w]

            # OCR
            plate_text = pytesseract.image_to_string(new_img, lang='eng')
            cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 3)
            break

    # Convert image (with contour) to Tkinter format
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(image_rgb)
    img_tk = ImageTk.PhotoImage(img_pil)

    # Show in Tkinter window
    image_label.configure(image=img_tk)
    image_label.image = img_tk

    # Show OCR result
    result_label.config(text=f"Detected Plate: {plate_text.strip()}")


# Upload Button
btn = Button(root, text="Upload Image & Detect", command=upload_and_detect, font=("Arial", 14))
btn.pack(pady=20)

root.mainloop()
